<a href="https://colab.research.google.com/github/demekeendalie/Fine-tuning-bert-model-for-amharic-news-classification/blob/main/Fine_tuning_BERT_for_Amharic_news_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf


2025-05-25 16:24:21.701299: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748190261.935150      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748190261.999836      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# reading the dataset from my drive using the above drive
dataframe1 =pd.read_excel('/kaggle/input/new-classification/dataset1.xlsx')
dataframe2 =pd.read_excel('/kaggle/input/new-classification/dataset2.xlsx')

In [3]:
dataframe1.category.unique()

array(['accident', 'agriculture', 'health', 'education', 'economy',
       'politics', 'sport'], dtype=object)

In [4]:
# merging the two dataframes
frame=[dataframe1,dataframe2]
dataframe=pd.concat(frame)

In [5]:
print(dataframe)

                                              articles   category
0     ግንቦት 13/2011  በተሽከርካሪ አደጋ ድንገተኛ ህክምና የተፈቀደው ሁ...   accident
1    ሚያዝያ  25/ 2011 የሰሜን ተራሮች ብሄራዊ ፓርክን የአደጋ ስጋቶች ለ...   accident
2    በደቡብ ወሎ ዞን ኩታበር ወረዳ ዛሬ በደረሰ የትራፊክ አደጋ በ11 ሰዎች ...   accident
3    የህዝብ ተወካዮች ምክር ቤት አባላት ዛሬ ጠዋት የተፈናቀሉ ወገኖችን ለመጎ...   accident
4    በባህር ዳር ከተማ ቀበሌ 05 በእሳት አደጋ ቤትና ንብረት የወደመባቸው ሰ...   accident
..                                                 ...        ...
695  ጎባ፤ መስከረም 16/2015 (ኢዜአ) መደ ወላቡ ዩኒቨርሲቲ ተጨማሪ 13 ...  education
696  ባህር ዳር ሰኔ 7/2010 በአማራ ክልል ከህብረተሰቡ ከደረሱት ጥቆማዎች ...  education
697  አዲስ አበባ ሰኔ 29/2010 የትምህርት ጥራት ጉድለት በአገሪቱ ላይ ዘር...  education
698  አዲስ አበባ፤ መስከረም 15/ 2016 (ኢዜአ)፦ በተባበሩት አረብ ኤምሬቶ...  education
699  አዲስ አበባ መጋቢት 4/2012(ኢዜአ) ግማሽ ቢሊዮን ብር ወጪ የተደረገበ...  education

[2181 rows x 2 columns]


In [6]:
# changing category to numeric
label=pd.Categorical(dataframe['category']).codes
label=pd.DataFrame(label)
dataframe['labels']=pd.DataFrame(label)
print(dataframe)

                                              articles   category  labels
0     ግንቦት 13/2011  በተሽከርካሪ አደጋ ድንገተኛ ህክምና የተፈቀደው ሁ...   accident       0
1    ሚያዝያ  25/ 2011 የሰሜን ተራሮች ብሄራዊ ፓርክን የአደጋ ስጋቶች ለ...   accident       0
2    በደቡብ ወሎ ዞን ኩታበር ወረዳ ዛሬ በደረሰ የትራፊክ አደጋ በ11 ሰዎች ...   accident       0
3    የህዝብ ተወካዮች ምክር ቤት አባላት ዛሬ ጠዋት የተፈናቀሉ ወገኖችን ለመጎ...   accident       0
4    በባህር ዳር ከተማ ቀበሌ 05 በእሳት አደጋ ቤትና ንብረት የወደመባቸው ሰ...   accident       0
..                                                 ...        ...     ...
695  ጎባ፤ መስከረም 16/2015 (ኢዜአ) መደ ወላቡ ዩኒቨርሲቲ ተጨማሪ 13 ...  education       3
696  ባህር ዳር ሰኔ 7/2010 በአማራ ክልል ከህብረተሰቡ ከደረሱት ጥቆማዎች ...  education       3
697  አዲስ አበባ ሰኔ 29/2010 የትምህርት ጥራት ጉድለት በአገሪቱ ላይ ዘር...  education       3
698  አዲስ አበባ፤ መስከረም 15/ 2016 (ኢዜአ)፦ በተባበሩት አረብ ኤምሬቶ...  education       3
699  አዲስ አበባ መጋቢት 4/2012(ኢዜአ) ግማሽ ቢሊዮን ብር ወጪ የተደረገበ...  education       3

[2181 rows x 3 columns]


In [7]:
# mixing each categories and thier labels
dataframe=dataframe.sample(frac=1)

In [8]:
dataframe['word_len'] = dataframe['articles'].str.split().str.len()
print(dataframe['word_len'].mean())

314.2563044475011


In [9]:
#number of news under each categories
#accident=dataframe[dataframe['category']=='accident']
#agir=dataframe[dataframe['category']=='agriculture']
#health=dataframe[dataframe['category']=='health']
#education=dataframe[dataframe['category']=='education']
#economy=dataframe[dataframe['category']=='economy']
#politics=dataframe[dataframe['category']=='politics']
#sport=dataframe[dataframe['category']=='sport']
#print(accident.shape)
#print(agir.shape)
#print(health.shape)
#print(education.shape)
#print(economy.shape)
#print(politics.shape)
#print(sport.shape)

In [10]:
#normalizarion
import re
# normalizatio table
def normalization(input_token):
    rep1=re.sub('[ሃኅኃሐሓኻ]','ሀ',input_token)
    rep2=re.sub('[ሑኁዅ]','ሁ',rep1)
    rep3=re.sub('[ኂሒኺ]','ሂ',rep2)
    rep4=re.sub('[ኌሔዄ]','ሄ',rep3)
    rep5=re.sub('[ሕኅ]','ህ',rep4)
    rep6=re.sub('[ኆሖኾ]','ሆ',rep5)
    rep7=re.sub('[ሠ]','ሰ',rep6)
    rep8=re.sub('[ሡ]','ሱ',rep7)
    rep9=re.sub('[ሢ]','ሲ',rep8)
    rep10=re.sub('[ሣ]','ሳ',rep9)
    rep11=re.sub('[ሤ]','ሴ',rep10)
    rep12=re.sub('[ሥ]','ስ',rep11)
    rep13=re.sub('[ሦ]','ሶ',rep12)
    rep14=re.sub('[ዓኣዐ]','አ',rep13)
    rep15=re.sub('[ዑ]','ኡ',rep14)
    rep16=re.sub('[ዒ]','ኢ',rep15)
    rep17=re.sub('[ዔ]','ኤ',rep16)
    rep18=re.sub('[ዕ]','እ',rep17)
    rep19=re.sub('[ዖ]','ኦ',rep18)
    rep20=re.sub('[ጸ]','ፀ',rep19)
    rep21=re.sub('[ጹ]','ፁ',rep20)
    rep22=re.sub('[ጺ]','ፂ',rep21)
    rep23=re.sub('[ጻ]','ፃ',rep22)
    rep24=re.sub('[ጼ]','ፄ',rep23)
    rep25=re.sub('[ጽ]','ፅ',rep24)
    rep26=re.sub('[ጾ]','ፆ',rep25)
    rep27=re.sub('(ሉ[ዋአ])','ሏ',rep26)
    rep28=re.sub('(ሙ[ዋአ])','ሟ',rep27)
    rep29=re.sub('(ቱ[ዋአ])','ቷ',rep28)
    rep30=re.sub('(ሩ[ዋአ])','ሯ',rep29)
    rep31=re.sub('(ሱ[ዋአ])','ሷ',rep30)
    rep32=re.sub('(ሹ[ዋአ])','ሿ',rep31)
    rep33=re.sub('(ቁ[ዋአ])','ቋ',rep32)
    rep34=re.sub('(ቡ[ዋአ])','ቧ',rep33)
    rep35=re.sub('(ቹ[ዋአ])','ቿ',rep34)
    rep36=re.sub('(ሁ[ዋአ])','ኋ',rep35)
    rep37=re.sub('(ኑ[ዋአ])','ኗ',rep36)
    rep38=re.sub('(ኙ[ዋአ])','ኟ',rep37)
    rep39=re.sub('(ኩ[ዋአ])','ኳ',rep38)
    rep40=re.sub('(ዙ[ዋአ])','ዟ',rep39)
    rep41=re.sub('(ጉ[ዋአ])','ጓ',rep40)
    rep42=re.sub('(ደ[ዋአ])','ዷ',rep41)
    rep43=re.sub('(ጡ[ዋአ])','ጧ',rep42)
    rep44=re.sub('(ጩ[ዋአ])','ጯ',rep43)
    rep45=re.sub('(ጹ[ዋአ])','ጿ',rep44)
    rep46=re.sub('(ፉ[ዋአ])','ፏ',rep45)
    rep47=re.sub('[ቊ]','ቁ',rep46) #ቁ can be written as ቊ
    rep48=re.sub('[ኵ]','ኩ',rep47) #ኩ can be also written as ኵ
    return rep48

In [11]:
# apply normalization and special characters removal
dataframe['articles']=dataframe['articles'].apply(lambda x: normalization(x))

In [12]:
# special symbol removal
dataframe['articles']=dataframe['articles'].str.replace('[^\w\s]','',regex=True)

In [13]:
# removing digits from the dataset
dataframe['articles']=dataframe['articles'].str.replace('\d+','',regex=True)

In [14]:
print(dataframe)

                                               articles     category  labels  \
931   አይ ኤም ኤፍ ኢትዮጵያ ባለፉት አስር አመታት አበረታች ኢኮኖሚያዊ እድገት...      economy       2   
295   በኢሏባቦር ዞን ከ ነጥብ  ሚሊዮን ኩንታል ምርት ተሰብስቧል\n\nመቱ ጥር...  agriculture       1   
20    በምስራቅ ሀረርጌ ዞን በደረሰ የተሽከርካሪ አደጋ የስድስት ሰዎች ህይወት ...     accident       0   
279   በአማራ ክልለ የተከሰተው የተምች ወረርሽኝ እንዳይዛመት ለመከላከል እየተሰ...  agriculture       1   
494   አምቦሆሳእና ጥር    ኢዜአ በምእራብ ሸዋ ዞን በአርሶ አደሩ ዘንድ እየተ...  agriculture       4   
...                                                 ...          ...     ...   
0     ብርሀን ፈይሳየኢትዮጵያ ቦክስ ፌዴሬሽን በየአመቱ የሚያዘጋጀው የክለቦች ቻ...        sport       0   
1161  ውሳኔው ሁለቱን ሀገራት ከፀጥታ ስጋት በማላቀቅ ኢኮኖሚያዊ ትስስር ለመፍጠ...     politics       5   
609   ዩኒሴፍ ለተንቀሳቃሽ ህክምና  ተሽከርካሪዎችን ለኢትዮጵያ ድጋፍ ሊሰጥ ነው...       health       4   
128   አዋሽ ባንክና አዋሽ ኢንሹራንስ  ሺህ ችግኞች ተከሉ     የአዋሽ ባንክና...      economy       0   
464   ባህር ዳር ጥር   ኢዜአ በሀገሪቱ በ ምርት ዘመን ለበልግና መኽር ወቅት ...  agriculture       4   

      word_len  
931        319  
295  

In [15]:
# train test split
from sklearn.model_selection import train_test_split
train_val_df, test_dataset = train_test_split(dataframe, test_size=0.20, random_state=42)
train_dataset, evaluation_dataset = train_test_split(train_val_df, test_size=0.10, random_state=42)
print('Training dataset shape: ', train_dataset.shape)
print('Validation dataset shape: ', evaluation_dataset.shape)
print('Testing dataset shape: ', test_dataset.shape)

Training dataset shape:  (1569, 4)
Validation dataset shape:  (175, 4)
Testing dataset shape:  (437, 4)


In [16]:
print(dataframe.head(10))

                                               articles     category  labels  \
931   አይ ኤም ኤፍ ኢትዮጵያ ባለፉት አስር አመታት አበረታች ኢኮኖሚያዊ እድገት...      economy       2   
295   በኢሏባቦር ዞን ከ ነጥብ  ሚሊዮን ኩንታል ምርት ተሰብስቧል\n\nመቱ ጥር...  agriculture       1   
20    በምስራቅ ሀረርጌ ዞን በደረሰ የተሽከርካሪ አደጋ የስድስት ሰዎች ህይወት ...     accident       0   
279   በአማራ ክልለ የተከሰተው የተምች ወረርሽኝ እንዳይዛመት ለመከላከል እየተሰ...  agriculture       1   
494   አምቦሆሳእና ጥር    ኢዜአ በምእራብ ሸዋ ዞን በአርሶ አደሩ ዘንድ እየተ...  agriculture       4   
224   ሳም ሳም አርጊኝ  ሳም ሳም አርጊኝ ያገርሽ ልጅ ነኝ ኢህአዴግና ብልፅግና...     politics       1   
793   የዩኒቨርስቲ ኢንዱስትሪ ትስስርን ማጠናከር እንደሚገባ ተመለከተ\n\nመቱ ...    education       3   
374   የምጣኔ ሀብት ችግሩን ለመፍታት የግብርናውን ዘርፍ ማዘመን የግድ መሆኑን ...  agriculture       1   
580   የኢትዮጵያ ህብረተሰብ ጤና ኢንስቲትዩት ወደ አገር ለሚገቡና ለሚወጡ ተጓዦ...       health       4   
1076  ሰራዊቱ ህገ መንግስታዊ ስርአቱን በማስጠበቅና ህዝባዊነቱን በማጠናከር ለአ...     politics       5   

      word_len  
931        319  
295        242  
20         197  
279        415  
494        377  
224        903  


In [17]:
msk = np.random.rand(len(dataframe)) < 0.8
train_dataset = dataframe[msk]
test_dataset = dataframe[~msk]

In [18]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 6.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.8.4.1 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cudnn-cu12==9.1.0.70; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cudnn-cu12 9.3.0.75 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cufft-cu12==11.2.1.3; platform_system == "Linux" and platform_machine == "x86_64", but you 

In [19]:
from datasets import Dataset

In [20]:
#convert format of the dataset to HuggingFace Dataset from Pandas DataFrame
test_dataset=Dataset.from_pandas(test_dataset)


In [21]:
#convert the format of the dataset to HuggingFace Dataset from Pandas DataFrame
train_dataset=Dataset.from_pandas(train_dataset)

In [22]:
#convert the format of the dataset to HuggingFace Dataset from Pandas DataFrame
evaluation_dataset=Dataset.from_pandas(evaluation_dataset)

In [23]:
#preview of the dataset after conversion
(test_dataset)

Dataset({
    features: ['articles', 'category', 'labels', 'word_len', '__index_level_0__'],
    num_rows: 442
})

In [24]:
print(test_dataset['labels'])

[1, 3, 2, 3, 5, 6, 0, 5, 3, 4, 2, 1, 4, 0, 3, 4, 5, 0, 4, 6, 0, 1, 5, 2, 3, 0, 0, 0, 1, 3, 5, 4, 3, 4, 3, 3, 6, 2, 4, 6, 0, 1, 3, 0, 1, 4, 2, 3, 4, 1, 6, 5, 2, 5, 1, 6, 4, 4, 5, 5, 6, 6, 3, 1, 2, 1, 1, 6, 4, 6, 5, 1, 1, 0, 4, 0, 6, 3, 4, 4, 5, 6, 0, 0, 6, 1, 1, 0, 4, 0, 1, 4, 5, 1, 2, 4, 4, 0, 3, 0, 3, 3, 5, 1, 3, 1, 2, 1, 2, 0, 0, 4, 1, 4, 4, 5, 5, 0, 0, 3, 1, 3, 4, 1, 4, 4, 0, 5, 6, 3, 0, 6, 3, 0, 4, 1, 1, 5, 2, 3, 6, 0, 4, 3, 6, 3, 1, 0, 1, 4, 1, 4, 4, 4, 0, 0, 4, 1, 6, 2, 4, 5, 0, 3, 4, 1, 1, 3, 4, 3, 4, 3, 0, 0, 3, 0, 1, 4, 4, 5, 0, 1, 2, 6, 4, 1, 4, 3, 4, 4, 1, 6, 6, 1, 3, 0, 0, 5, 5, 2, 6, 0, 2, 1, 5, 1, 4, 2, 2, 0, 0, 5, 4, 6, 0, 0, 2, 4, 1, 1, 6, 5, 0, 0, 1, 2, 5, 6, 1, 6, 6, 0, 3, 0, 2, 0, 0, 3, 5, 1, 0, 5, 1, 0, 4, 6, 0, 1, 6, 1, 4, 2, 6, 3, 0, 3, 4, 0, 1, 2, 0, 0, 6, 3, 1, 1, 3, 2, 5, 3, 0, 1, 4, 6, 1, 0, 0, 4, 4, 1, 2, 5, 2, 4, 1, 5, 0, 2, 4, 5, 4, 4, 6, 4, 6, 0, 6, 3, 0, 3, 0, 1, 5, 0, 1, 4, 1, 0, 5, 1, 0, 2, 1, 4, 0, 1, 6, 4, 2, 4, 6, 0, 4, 4, 0, 2, 3, 0, 5, 0, 4, 3, 2, 

In [25]:
(train_dataset)

Dataset({
    features: ['articles', 'category', 'labels', 'word_len', '__index_level_0__'],
    num_rows: 1739
})

In [26]:
(evaluation_dataset)

Dataset({
    features: ['articles', 'category', 'labels', 'word_len', '__index_level_0__'],
    num_rows: 175
})

In [27]:
#remove unnecessary column
test_dataset=test_dataset.remove_columns("__index_level_0__")
train_dataset=train_dataset.remove_columns("__index_level_0__")
evaluation_dataset=evaluation_dataset.remove_columns("__index_level_0__")

In [28]:
#combine the train and test dataset into one datset
import datasets
main_dataset= datasets.DatasetDict({
    'train': train_dataset,
    'test': test_dataset,
    'evaluate': evaluation_dataset
})

In [29]:
main_dataset

DatasetDict({
    train: Dataset({
        features: ['articles', 'category', 'labels', 'word_len'],
        num_rows: 1739
    })
    test: Dataset({
        features: ['articles', 'category', 'labels', 'word_len'],
        num_rows: 442
    })
    evaluate: Dataset({
        features: ['articles', 'category', 'labels', 'word_len'],
        num_rows: 175
    })
})

In [30]:
# training and testing data size
training_data_size = main_dataset['train'].num_rows
testing_data_size = main_dataset['test'].num_rows
evaluation_data_size = main_dataset['evaluate'].num_rows

In [31]:
from transformers import AutoTokenizer

In [32]:
#loading a tokenizer from the pretrained model
tokenizer = AutoTokenizer.from_pretrained("Davlan/bert-base-multilingual-cased-finetuned-amharic")

tokenizer_config.json:   0%|          | 0.00/335 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/798 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.55M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [33]:
#Have a tokenizer function that uses the tokenizer
def tokenize_function(data):
    return tokenizer(data["articles"], padding="max_length", truncation=True)

In [34]:
#Tokenize all the data using the mapping functionality
tokenized_datasets = main_dataset.map(tokenize_function)

Map:   0%|          | 0/1739 [00:00<?, ? examples/s]

Map:   0%|          | 0/442 [00:00<?, ? examples/s]

Map:   0%|          | 0/175 [00:00<?, ? examples/s]

In [35]:
#remove the posts column as it is no longer needed
tokenized_datasets = tokenized_datasets.remove_columns(["articles"])

In [36]:
#import torch
import torch
#import Data loader from torch
from torch.utils.data import DataLoader
#import an optimizer
from torch.optim import AdamW
#import tqdm for a progress bar
from tqdm.auto import tqdm

In [37]:
#changing the format of the tokenized dataset to torch
tokenized_datasets.set_format("torch")

In [38]:
#shuffeling and selecting the needed size of dataset for training and evaluating the model
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(training_data_size))
small_test_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(testing_data_size))
small_eval_dataset = tokenized_datasets["evaluate"].shuffle(seed=42).select(range(evaluation_data_size))

In [39]:
# preview of the shuffeled and selected evaluation dataset
small_eval_dataset

Dataset({
    features: ['category', 'labels', 'word_len', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 175
})

In [40]:
train_dataloader = DataLoader(small_train_dataset, shuffle=True, batch_size=4)
eval_dataloader = DataLoader(small_eval_dataset, batch_size=4)
test_dataloader = DataLoader(small_test_dataset, batch_size=4)

In [41]:
#Load auto mode classifier from the pretrained model
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("Davlan/bert-base-multilingual-cased-finetuned-amharic", num_labels=7)

pytorch_model.bin:   0%|          | 0.00/712M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/712M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at Davlan/bert-base-multilingual-cased-finetuned-amharic and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [42]:
import torch

device = torch.device("cpu")

if torch.cuda.is_available():
   print("Training on GPU")
   device = torch.device("cuda:0")
print(device)

cpu


In [43]:
!pip install evaluate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.2 MB/s eta 0:00:00


In [44]:
import numpy as np
from evaluate import load
metric = load("f1","accuracy")

In [45]:

#def compute_metrics(eval_pred):
 #   logits, labels = eval_pred
#    predictions = np.argmax(logits, axis=-1)
 #   return metric.compute(predictions=predictions, references=labels, average="weighted")

In [46]:
# #load an optimizer
# optimizer = AdamW(model.parameters(), lr=5e-5)
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

In [47]:
from transformers import TrainingArguments, Trainer
from transformers import EarlyStoppingCallback, IntervalStrategy

In [48]:
def custom_metrics(eval_pred):
    metric1 = load("precision")
    metric2 = load("recall")
    metric3 = load("f1")
    metric4 = load("accuracy")

    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    precision = metric1.compute(predictions=predictions, references=labels, average="weighted")["precision"]
    recall = metric2.compute(predictions=predictions, references=labels, average="weighted")["recall"]
    f1 = metric3.compute(predictions=predictions, references=labels, average="weighted")["f1"]
    accuracy = metric4.compute(predictions=predictions, references=labels)["accuracy"]

    return {"precision": precision, "recall": recall, "f1": f1, "accuracy": accuracy}

In [49]:
# #load an optimizer
# optimizer = AdamW(model.parameters(), lr=5e-5)
training_args = TrainingArguments(
   output_dir="./results",
   eval_strategy='epoch',
   save_strategy='epoch',
   logging_strategy='epoch',
   num_train_epochs=10,
   learning_rate=1e-5,
   per_device_train_batch_size=4,  # batch size per device during training
   per_device_eval_batch_size=8,   # batch size for evaluation
   warmup_steps=1000,                # number of warmup steps for learning rate
   weight_decay=0.01,
   run_name="Amharic news document",# strength of weight decay
   logging_dir='./logs',            # directory for storing logs
   logging_steps=20,
   load_best_model_at_end= True,

   )

In [50]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=custom_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=10)],
)

In [51]:
import torch
torch.cuda.empty_cache()


In [52]:
# checking the model performance by using k-fold
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments, AutoTokenizer
from sklearn.model_selection import KFold

In [53]:
X_train=train_val_df['articles']
y_train=train_val_df['labels']

In [ ]:
train_result=trainer.train()

<IPython.core.display.Javascript object>

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import matplotlib.pyplot as plt

# Assuming you have stored the losses during training in these lists
data=pd.DataFrame(trainer.state.log_history)
print(data.head(10))
data=data.iloc[:200]
train_losses=data['loss']
eval_losses=data['eval_loss']
epoch=data['epoch']
# Plotting
plt.figure(figsize=(10, 5))
plt.plot(epoch,train_losses, label='Training Loss', color='red')
plt.plot(epoch, eval_losses, label='Validation Loss', color='blue')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid()
plt.show()

In [ ]:

from sklearn.metrics import confusion_matrix
# Now you can make predictions
predictions = trainer.predict(small_test_dataset)

# Get the predicted labels and true labels
predicted_labels = np.argmax(predictions.predictions, axis=1)
true_labels =  small_test_dataset['labels']
# Calculate the confusion matrix
cm = confusion_matrix(true_labels, predicted_labels)
print(cm)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(true_labels, predicted_labels))

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.show()

In [ ]:
#overall performance of the fine-tuned BERT model on Amharic new classification
trainer.evaluate(small_test_dataset)

# New Section